##### Reference: https://github.com/deepseasw/seq2seq_chatbot

In [1]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing
from tensorflow.keras.utils import Sequence

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Mecab

## 데이터 로드

Seq2Seq에서의 임베딩이 이전 예제와 다른 점은 아래와 같이 태그를 사용한다는 것입니다. 임베딩의 0~3번째에 각각 PADDING, START, END, OOV 태그를 넣습니다. 사실 그냥 똑같은 단어라고 보시면 됩니다. 다만 이 단어들이 Seq2Seq의 동작을 제어합니다. 

예를 들어, 디코더 입력에 START가 들어가면 디코딩의 시작을 의미합니다. 반대로 디코더 출력에 END가 나오면 디코딩을 종료합니다.

In [2]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

# 챗봇 데이터 로드
chatbot_data = pd.read_csv('./dataset/chatbot/ChatbotData.csv', encoding='utf-8')
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

챗봇의 훈련을 위해서 송영숙님이 공개한 한글 데이터셋을 로드합니다. 질문과 대답, 감정 등 총 3개의 항목으로 구성되어 있습니다. 감정 분류는 Seq2Seq에 필요가 없기 때문에 사용하지 않습니다.

https://github.com/songys/Chatbot_data
<br>
<br>
<br>

In [3]:
# 데이터 개수
len(question)

11823

```
# 데이터의 일부만 학습에 사용
question = question[:100]
answer = answer[:100]

# 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()
```

In [4]:
# 데이터 전부 학습에 사용!!
# 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 12시 땡!
A : 하루가 또 가네요.

Q : 1지망 학교 떨어졌어
A : 위로해 드립니다.

Q : 3박4일 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : 3박4일 정도 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : PPL 심하네
A : 눈살이 찌푸려지죠.

Q : SD카드 망가졌어
A : 다시 새로 사는 게 마음 편해요.

Q : SD카드 안돼
A : 다시 새로 사는 게 마음 편해요.

Q : SNS 맞팔 왜 안하지ㅠㅠ
A : 잘 모르고 있을 수도 있어요.

Q : SNS 시간낭비인 거 아는데 매일 하는 중
A : 시간을 정하고 해보세요.

Q : SNS 시간낭비인데 자꾸 보게됨
A : 시간을 정하고 해보세요.



## 단어 사전 생성

In [5]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Mecab()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

문장을 먼저 최소 단위인 토큰으로 나누어야 합니다. 한글 문장이라면 보통 KoNLPy(코엔엘파이)를 통해 형태소분석을 적용합니다. Mecab 외에 Hannanum, Kkma, Komoran, Mecab, Okt(Twitter) 등 여러가지 라이브러리를 동일한 인터페이스로 실행이 가능합니다.

KoNLPy는 자바로 구현이 되어 있습니다. 맥OS와 달리 윈도우 아나콘다는 아래와 같은 작업을 거쳐야 합니다.

< 1. 자바 설치 ><br>
커맨드창에서 'java -version' 명령어를 실행하여 버전이 1.7 이상이어야 합니다.

< 2. JPype1 설치 ><br>
파이썬에서 자바를 호출할 수 있는 라이브러리입니다.

< 3. KoNLPy 설치 ><br>
위의 두 가지가 미리 완료되어 있어야만 가능합니다.

아래 링크에서 좀 더 자세한 방법을 확인하시기 바랍니다.<br>
-> https://ericnjennifer.github.io/python_visualization/2018/01/21/PythonVisualization_Chapt1.html
<br>
<br>
<br>

In [6]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 12 시 땡
A : 하루 가 또 가 네요

Q : 1 지망 학교 떨어졌 어
A : 위로 해 드립니다

Q : 3 박 4 일 놀 러 가 고 싶 다
A : 여행 은 언제나 좋 죠

Q : 3 박 4 일 정도 놀 러 가 고 싶 다
A : 여행 은 언제나 좋 죠

Q : PPL 심하 네
A : 눈살 이 찌푸려 지 죠

Q : SD 카드 망가졌 어
A : 다시 새로 사 는 게 마음 편해요

Q : SD 카드 안 돼
A : 다시 새로 사 는 게 마음 편해요

Q : SNS 맞 팔 왜 안 하 지 ㅠㅠ
A : 잘 모르 고 있 을 수 도 있 어요

Q : SNS 시간 낭비 인 거 아 는데 매일 하 는 중
A : 시간 을 정하 고 해 보 세요

Q : SNS 시간 낭비 인데 자꾸 보 게 됨
A : 시간 을 정하 고 해 보 세요



In [7]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

질문과 대답 문장들을 합쳐서 전체 단어 사전을 만듭니다. 자연어처리에서는 항상 이렇게 단어를 인덱스에 따라 정리를 해야 합니다. 그래야지 문장을 인덱스 배열로 바꿔서 임베딩 레이어에 넣을 수 있습니다. 또한 모델의 출력에서 나온 인덱스를 다시 단어로 변환하는데도 필요합니다.
<br>
<br>
<br>

In [8]:
# 단어 개수
len(words)

6852

In [9]:
# 단어 출력
words[:20]

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '설마',
 '그런지',
 '마냥',
 '아재',
 '멋',
 '내편',
 '강림',
 '관계',
 '소분',
 '드나',
 '둬서',
 '민감',
 '중요',
 '갔',
 '쓰디쓰',
 '성적']

In [10]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [11]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
dict(list(word_to_index.items())[:20])

{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '설마': 4,
 '그런지': 5,
 '마냥': 6,
 '아재': 7,
 '멋': 8,
 '내편': 9,
 '강림': 10,
 '관계': 11,
 '소분': 12,
 '드나': 13,
 '둬서': 14,
 '민감': 15,
 '중요': 16,
 '갔': 17,
 '쓰디쓰': 18,
 '성적': 19}

In [12]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
dict(list(index_to_word.items())[:20])

{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '설마',
 5: '그런지',
 6: '마냥',
 7: '아재',
 8: '멋',
 9: '내편',
 10: '강림',
 11: '관계',
 12: '소분',
 13: '드나',
 14: '둬서',
 15: '민감',
 16: '중요',
 17: '갔',
 18: '쓰디쓰',
 19: '성적'}

## 전처리

In [13]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

Seq2Seq에서는 학습시 다음과 같이 총 3개의 데이터가 필요합니다.

인코더 입력 : 12시 땡<br>
디코더 입력 : START 하루 가 또 가네요<br>
디코더 출력 : 하루 가 또 가네요 END

원래 Seq2Seq는 디코더의 현재 출력이 디코더의 다음 입력으로 들어갑니다. 다만 학습에서는 굳이 이렇게 하지 않고 디코더 입력과 디코더 출력의 데이터를 각각 만듭니다. 

그러나 예측시에는 이런 방식이 불가능합니다. 출력값을 미리 알지 못하기 때문에, 디코더 입력을 사전에 생성할 수가 없습니다. 이런 문제를 해결하기 위해 훈련 모델과 예측 모델을 따로 구성해야 합니다. 모델 생성 부분에서 다시 자세히 설명을 드리겠습니다.

In [14]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (12시 땡)
x_encoder[0]

array([1854, 2677, 3998,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [15]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (START 하루 가 또 가네요)
x_decoder[0]

array([   1, 6600, 4109, 4831, 4109, 6628,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [16]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
y_decoder[0]

array([6600, 4109, 4831, 4109, 6628,    2,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

Memory explosion을 방지하기 위해 `fit_generator` 매서드를 위한 class를 구성합니다.

In [17]:
class BatchSequence(Sequence):
    def __init__(self, x_encoder, x_decoder, y_decoder):
        self.x_encoder = x_encoder
        self.x_decoder = x_decoder
        self.y_decoder = y_decoder
        self.batch_size = 128

    def __len__(self):
        return len(self.x_encoder) // self.batch_size

    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = start + self.batch_size

        # one-hot encoding
        x = self.x_encoder[start:end]
        y = self.x_decoder[start:end]
        target = self.y_decoder[start:end]
        
        one_hot_data = np.zeros((len(target), max_sequences, len(words)))

        # 디코더 목표를 원핫인코딩으로 변환
        # 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
        for i, sequence in enumerate(target):
            for j, index in enumerate(sequence):
                one_hot_data[i, j, index] = 1

        # 디코더 목표 설정
        target = one_hot_data

        return [x, y], target

인코더 입력과 디코더 입력은 임베딩 레이어에 들어가는 인덱스 배열입니다. 반면에 디코더 출력은 원핫인코딩 형식이 되어야 합니다. 디코더의 마지막 Dense 레이어에서 softmax로 나오기 때문입니다.

## 모델 생성

In [18]:
#--------------------------------------------
# 훈련 모델 인코더 정의                       
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])    

지금까지의 예제는 Sequential 방식의 모델이었습니다. 하지만 이번에는 함수형 API 모델을 사용했습니다. 인코더와 디코더가 따로 분리되어야 하는데, 단순히 레이어를 추가하여 붙이는 순차형으로는 구현이 불가능하기 때문입니다. 

Model() 함수로 입력과 출력을 따로 설정하여 모델을 만듭니다. 그다음 compile과 fit은 이전과 동일하게 적용하시면 됩니다.
<br>
<br>
<br>

In [19]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)


#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용합니다. 예측 모델 인코더는 훈련 모델 인코더과 동일합니다. 그러나 예측 모델 디코더는 매번 LSTM 상태값을 입력으로 받습니다. 또한 디코더의 LSTM 상태를 출력값과 같이 내보내서, 다음 번 입력에 넣습니다. 

이렇게 하는 이유는 LSTM을 딱 한번의 타임 스텝만 실행하기 때문입니다. 그래서 매번 상태값을 새로 초기화 해야 합니다. 이와 반대로 훈련할때는 문장 전체를 계속 LSTM으로 돌리기 때문에 자동으로 상태값이 전달됩니다. 

## 훈련 및 테스트

In [20]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [21]:
seq = BatchSequence(x_encoder, x_decoder, y_decoder)
len(seq)

92

In [22]:
convert_index_to_text(x_encoder[2], index_to_word)

'3 박 4 일 놀 러 가 고 싶 다 <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> '

In [23]:
convert_index_to_text(x_decoder[2], index_to_word)

'<START> 여행 은 언제나 좋 죠 <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> '

In [24]:
%%time

# 에폭 반복
for epoch in range(10):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit_generator(seq,
                                  epochs=100,
                                  workers=6,
                                  verbose=0)

    # 정확도와 손실 출력
    print(history.history)
    print('accuracy :', history.history['accuracy'][-1])
    print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    # (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()

Total Epoch : 1
{'loss': [3.541323652733927, 1.7396523447140404, 1.5714846991974374, 1.4982836155787758, 1.4432028350622759, 1.3971302444520204, 1.3577766729437786, 1.3256785247636877, 1.2963844459989797, 1.2681558054426443, 1.2417016489350277, 1.2144042823625647, 1.1869707444439763, 1.1610628243373788, 1.137887970260952, 1.1175076870814613, 1.0982828431803247, 1.0802074748536814, 1.062230689370114, 1.0465151110421056, 1.0305719278428866, 1.016612301054208, 1.003957754243975, 0.991715435748515, 0.9806103486081829, 0.9690482415582823, 0.9590207584526228, 0.9491946995258331, 0.9395164244848749, 0.9312361673168514, 0.9220827098773874, 0.9140815676554389, 0.9057094383498897, 0.8986951592175857, 0.8908465316762095, 0.8837911920702975, 0.876490091500075, 0.8698211636232294, 0.8630648786606996, 0.8568725475798482, 0.8500056318614794, 0.8433882645938707, 0.837526754192684, 0.8317627602297327, 0.8256434565005095, 0.8199616657650989, 0.8145555529905402, 0.809163717472035, 0.8030636790006057, 0.7

{'loss': [0.32858292742267897, 0.32768267441702925, 0.3267730077647645, 0.324639140587786, 0.32424516133640124, 0.3235150361838548, 0.3218792027429394, 0.32058547939295357, 0.3197461637141912, 0.31759721494239307, 0.3172932103600191, 0.3160459157889304, 0.31523717956050584, 0.3146335519850254, 0.31339028817803966, 0.3120879885619101, 0.3103091149874356, 0.30942148342728615, 0.30784704746759456, 0.30781503118898557, 0.3064639940857887, 0.30422996373280237, 0.30389785264497216, 0.30268416165009787, 0.3016317323174166, 0.30005157253016596, 0.3001830399684284, 0.2989211573225001, 0.2971497529550739, 0.29623557530019595, 0.2943846508860588, 0.2937918253566908, 0.2924216378318227, 0.29201519586469815, 0.2906162210780641, 0.2887384063847687, 0.2878670177381972, 0.28710286286862, 0.28656708724472835, 0.28452472307759785, 0.2834010001109994, 0.2825942251669324, 0.2817802983133689, 0.2794242081758769, 0.27897240545438684, 0.27802760639916296, 0.27660961060420325, 0.27613396975009336, 0.274242110

{'loss': [0.12115713540950547, 0.12073951558736355, 0.12059599197591128, 0.1197612221636202, 0.12049703352639209, 0.11948253473509914, 0.12042666599154472, 0.11993563312875188, 0.11877088167745134, 0.11891821314297292, 0.11833197846198859, 0.11735964380204678, 0.11665182589026897, 0.1173143510747215, 0.11662360612788926, 0.1160451899809034, 0.11666667967548837, 0.11606043670326471, 0.11515027086209992, 0.1150719029828906, 0.11576448036762683, 0.11448533136559569, 0.11464910638397155, 0.11469649539693542, 0.11367967684307824, 0.11388191293277171, 0.11325336219337971, 0.1125578758833201, 0.11210873646094748, 0.1126664207600381, 0.11252371221780777, 0.11176847010526968, 0.11143224245018285, 0.11060210329520961, 0.11065019825068505, 0.11068195149140514, 0.11060572958186916, 0.11047833627494781, 0.10990505317307037, 0.10902916968030774, 0.10988745595449986, 0.10892453862597114, 0.10890168441540521, 0.10837333264720181, 0.10864243036864893, 0.10784609636048907, 0.10832709426303273, 0.1078044

{'loss': [0.06225042796248327, 0.061126549252430384, 0.06184562012229277, 0.061540761274164135, 0.0609439079163839, 0.06078383973394723, 0.0610976036152114, 0.062079101983133864, 0.06142394645544498, 0.06086263809677051, 0.06053202433268661, 0.060675436995275646, 0.06106294634873453, 0.06061234740216447, 0.060260610746057784, 0.0606402861683265, 0.06044297575262254, 0.05936581342805015, 0.061008892244780843, 0.059935997867875776, 0.05967325089580339, 0.0598648385974862, 0.05938885261750092, 0.06006458063569406, 0.05933727704636429, 0.05921030658013795, 0.05872379741428987, 0.058581514122045555, 0.0590156185562196, 0.05887597352873696, 0.059007493779063225, 0.058276979212203754, 0.058818682856128915, 0.0583018916355365, 0.058050724434787815, 0.05811893304243036, 0.05794606410452853, 0.058842878626740494, 0.0586617774249095, 0.05768059140435704, 0.057500426748605525, 0.05761350490881697, 0.05742386906691219, 0.05676329625851434, 0.05735754960661997, 0.057534265767216035, 0.05797226356504

학습이 진행될수록 예측 문장이 제대로 생성되는 것을 볼 수 있습니다. 다만 여기서의 예측은 단순히 테스트를 위한 것이라, 인코더 입력과 디코더 입력 데이터가 동시에 사용됩니다. 아래 문장 생성에서는 예측 모델을 적용하기 때문에, 오직 인코더 입력 데이터만 집어 넣습니다.

In [29]:
model.save('test.h5')

```
# 에폭 반복
for epoch in range(20):
    print('Total Epoch :', epoch + 1)
```
> 이 부분이 왜 있는지 모르겠음... 하이퍼 파라미터 튜닝을 위한 것인가?

## 문장 생성

In [25]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [26]:
%%time

# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


제일 첫 단어는 START로 시작합니다. 그리고 출력으로 나온 인덱스를 디코더 입력으로 넣고 다시 예측을 반복합니다. 상태값을 받아 다시 입력으로 같이 넣는 것에 주의하시기 바랍니다. END 태그가 나오면 문장 생성을 종료합니다.
<br>
<br>
<br>

In [27]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('3박4일 놀러가고 싶다')
input_seq

array([[ 519, 4737, 5679,  239, 1024, 5110, 4109, 5340, 4473, 6085,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [31]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'여행 은 언제나 좋 죠 '

데이터셋에 있는 문장과 똑같은 입력을 넣으니, 역시 정확히 일치하는 답변이 출력되었습니다.
<br>
<br>
<br>

In [32]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('3박4일 같이 놀러가고 싶다')
input_seq

array([[ 519, 4737, 5679,  239, 6015, 1024, 5110, 4109, 5340, 4473, 6085,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [33]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'여행 은 언제나 좋 죠 '

데이터셋 문장에서는 없던 '같이'라는 단어를 추가해 보았습니다. 그래도 비슷한 의미란 것을 파악하여 동일한 답변이 나왔습니다.
<br>
<br>
<br>

In [34]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('3박4일 놀러가려고')
input_seq

array([[ 519, 4737, 5679,  239, 1024, 5110, 4109, 6479,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [35]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'여행 은 언제나 좋 죠 '

하지만 데이터셋에 없던 '가려고'로 입력을 수정하니, 전혀 다른 문장이 출력되었습니다. 이는 우리가 데이터의 일부인 100개 문장만 학습했기 때문입니다. 데이터의 개수를 늘려서 훈련할수록 일반화 능력이 더욱 높아집니다.

In [43]:
input_text = "위로 좀 해줘"
input_seq = make_predict_input(input_text)
generate_text(input_seq)

'제게 기대 세요 '

In [44]:
def answer(input_text):
    input_seq = make_predict_input(input_text)
    answer = generate_text(input_seq)
    return answer

In [54]:
answer("살기 힘드네ㅠ")

'그럼 요 한 순간 에 내 삶 속 에서 내보내 기 는 쉽 지 않 을 거 예요 '

In [55]:
answer("으엥ㅠㅠ 고마워")

'힘내 세요 '